<a href="https://colab.research.google.com/github/sku1978/sk-share-repo/blob/main/Spark/SparkDataFrame/SparkDataFrameNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq  > /dev/null 
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz > /dev/null 
!pip install -q findspark

In [25]:
!mkdir /content/conf /content/lib
!wget -O /content/conf/log4j.properties https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/log4j.properties > /dev/null 2>&1
!mv /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf.bk  > /dev/null 2>&1
!wget -O /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/spark-defaults.conf  > /dev/null 2>&1
!wget -O /content/conf/spark.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/spark.conf > /dev/null 2>&1

!wget -O /content/lib/logger.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/lib/logger.py  > /dev/null 2>&1
!wget -O /content/lib/utils.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/lib/utils.py  > /dev/null 2>&1

mkdir: cannot create directory ‘/content/conf’: File exists
mkdir: cannot create directory ‘/content/lib’: File exists


In [26]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.1-bin-hadoop3.2'

**Spark UI section**
<br>To use Spark UI, uncomment below sections and use the public link

In [4]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
#get_ipython().system_raw('./ngrok http 4050 &')

In [5]:
#!curl -s http://localhost:4040/api/tunnels

**Main Section**

In [27]:
from pyspark.sql import *
from pyspark import SparkConf, SparkFiles
from lib.logger import Log4J
from lib.utils import get_spark_app_config

conf=get_spark_app_config()

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

logger = Log4J(spark)

**Basic CSV Read**

In [7]:
def load_csv_file(spark, url):
   spark.sparkContext.addFile(url)

   survey_df=spark.read \
   .format("csv") \
   .option("header", "true") \
   .option("inferSchema", "true") \
   .option("mode", "FAILFAST") \
   .load('file://'+SparkFiles.get("sample.csv"))

   return survey_df

def count_by_country(survey_df):
  count_df= survey_df.select("Age", "Gender", "Country", "State") \
                     .where("Age <= 40") \
                     .groupBy("Country") \
                     .count()
  return count_df

In [8]:
logger.info("Start CSV Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/sample.csv'

survey_df=load_csv_file(spark, url)
partitioned_survey_df=survey_df.repartition(2)

count_df=count_by_country(partitioned_survey_df)

logger.info(count_df.collect())

logger.info("End CSV Load")

**Data Types**<br>
(https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.types)

**CSV Read (CSV)**

In [23]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType

logger.info("Start Flight Time CSV Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.csv'

spark.sparkContext.addFile(url)

flightSchemaStruct = StructType([StructField("FL_DATE", DateType(), True),
                                 StructField("OP_CARRIER", StringType(), True),
                                 StructField("OP_CARRIER_FL_NUM", IntegerType(), True),
                                 StructField("ORIGIN", StringType(), True),
                                 StructField("ORIGIN_CITY_NAME", StringType(), True),
                                 StructField("DEST", StringType(), True),
                                 StructField("DEST_CITY_NAME", StringType(), True),
                                 StructField("CRS_DEP_TIME", IntegerType(), True),
                                 StructField("DEP_TIME", IntegerType(), True),
                                 StructField("WHEELS_ON", IntegerType(), True),
                                 StructField("TAXI_IN", IntegerType(), True),
                                 StructField("CRS_ARR_TIME", IntegerType(), True),
                                 StructField("ARR_TIME", IntegerType(), True),
                                 StructField("CANCELLED", IntegerType(), True),
                                 StructField("DISTANCE", StringType(), True),
                                ])

flight_time_df=spark.read \
                    .format("csv") \
                    .option("header", "true") \
                    .option("mode", "FAILFAST") \
                    .option("dateFormat", "M/d/y") \
                    .schema(flightSchemaStruct) \
                    .load('file://'+SparkFiles.get("flight-time.csv"))

flight_time_df.show()

logger.info("End CSV Load")

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

**JSON Read**

In [28]:
logger.info("Start Flight Time JSON Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.json'

spark.sparkContext.addFile(url)

flightSchemaDDL = """FL_DATE DATE, 
                     OP_CARRIER STRING, 
                     OP_CARRIER_FL_NUM INT,
                     ORIGIN STRING,
                     ORIGIN_CITY_NAME STRING,
                     DEST STRING,
                     DEST_CITY_NAME STRING,
                     CRS_DEP_TIME INT,
                     DEP_TIME INT,
                     WHEELS_ON INT,
                     TAXI_IN INT,
                     CRS_ARR_TIME INT,
                     ARR_TIME INT,
                     CANCELLED INT,
                     DISTANCE STRING"""

flight_time_df=spark.read \
                    .format("json") \
                    .option("dateFormat", "M/d/y") \
                    .schema(flightSchemaDDL) \
                    .load('file://'+SparkFiles.get("flight-time.json"))

flight_time_df.show()

logger.info("End JSON Load")

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

**Parquet Read**

In [12]:
logger.info("Start Flight Time Parquet Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.parquet'

spark.sparkContext.addFile(url)

flight_time_df=spark.read \
                    .format("parquet") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("flight-time.parquet"))

flight_time_df.show()

logger.info("End Parquet Load")

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

**Write Avro file**<br>
Added <br>
spark.jars.packages                org.apache.spark:spark-avro_2.12:3.0.0
<br>to<br>
conf/spark-defaults.conf

In [33]:
logger.info("Start Flight Time Parquet Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.parquet'

spark.sparkContext.addFile(url)

flight_time_df=spark.read \
                    .format("parquet") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("flight-time.parquet"))

logger.info("End Parquet Load")

logger.info("Start Avro write")
logger.info("Number of partitions: " + str(flight_time_df.rdd.getNumPartitions()))

flight_time_df.write \
              .format("avro") \
              .mode("overwrite") \
              .option("path", "dataSink/avro/") \
              .save()

logger.info("End Avro write")

AnalysisException: ignored

2

**View Log**

In [29]:
!cat app-logs/sparklog.log

21/04/08 16:33:03 INFO Hello World: Start CSV Load
21/04/08 16:33:14 INFO Hello World: [[United Kingdom, 1], [Canada, 2], [United States, 4]]
21/04/08 16:33:14 INFO Hello World: End CSV Load
21/04/08 16:33:14 INFO Hello World: Start Flight Time CSV Load
21/04/08 16:33:20 INFO Hello World: End CSV Load
21/04/08 16:34:55 INFO Hello World: Start Flight Time JSON Load
21/04/08 16:35:01 INFO Hello World: End JSON Load
21/04/08 16:37:03 INFO Hello World: Start Flight Time Parquet Load
21/04/08 16:37:06 INFO Hello World: End JSON Load
21/04/08 16:45:12 INFO Hello World: Start Flight Time CSV Load
21/04/08 16:45:15 INFO Hello World: End CSV Load
21/04/08 17:08:47 INFO Hello World: Start Flight Time CSV Load
21/04/08 17:11:48 INFO Hello World: Start Flight Time CSV Load
21/04/08 17:12:23 INFO Hello World: Start Flight Time CSV Load
21/04/08 17:12:23 INFO Hello World: End CSV Load
21/04/08 17:12:49 INFO Hello World: Start Flight Time CSV Load
21/04/08 17:12:49 INFO Hello World: End CSV Load
21/0